In [1]:
# Task 1
from queue import PriorityQueue

def heuristic(current_pos, end_pos):
    return abs(current_pos[0] - end_pos[0]) + abs(current_pos[1] - end_pos[1])

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.h = 0  
        self.f = 0  

    def __lt__(self, other):
        return self.f < other.f

def best_first_search(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    
    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()
    
    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position
        
        if current_pos == end:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]
        
        visited.add(current_pos)
        
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if (0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and
                    maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited):
                new_node = Node(new_pos, current_node)
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h
                frontier.put(new_node)
                visited.add(new_pos)
    return None

def find_all_pairs_shortest_paths(maze, points):
    paths = {}
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            path = best_first_search(maze, points[i], points[j])
            if path:
                paths[(points[i], points[j])] = path
                paths[(points[j], points[i])] = path[::-1]
    return paths

def greedy_tsp(maze, start, goals):
    all_points = [start] + goals
    paths = find_all_pairs_shortest_paths(maze, all_points)
    
    current = start
    visited = set()
    ordered_path = []
    
    while len(visited) < len(goals):
        visited.add(current)
        ordered_path.append(current)

        # Get the next closest goal with a valid path
        next_goal = min(
            (goal for goal in goals if goal not in visited and (current, goal) in paths),
            key=lambda x: len(paths[(current, x)]),
            default=None
        )

        if next_goal is None:  # No reachable goal left
            break

        ordered_path.extend(paths[(current, next_goal)][1:])  # Avoid duplicate points
        current = next_goal
    
    return ordered_path if len(visited) == len(goals) else None


maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
]
start = (0, 0)
goals = [(4, 4), (1, 1), (0, 4), (4, 3)]

path = greedy_tsp(maze, start, goals)
if path:
    print("Path covering all goals:", path)
else:
    print("No path found")


Path covering all goals: [(0, 0), (1, 0), (1, 1), (1, 1), (1, 2), (1, 3), (0, 3), (0, 4), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (4, 4)]


In [2]:
# Task 2
import random
import time

graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'D': 4, 'E': 3},
    'C': {'F': 1, 'G': 5},
    'D': {'H': 2},
    'E': {},
    'F': {'I': 6},
    'G': {},
    'H': {},
    'I': {}
}

heuristic = {
    'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 7,
    'F': 3, 'G': 6, 'H': 2, 'I': 0
}

def update_edge_costs(graph):
    for node in graph:
        for neighbor in graph[node]:
            graph[node][neighbor] = random.randint(1, 10)

def a_star(graph, start, goal):
    frontier = [(start, heuristic[start])]
    g_costs = {start: 0}
    came_from = {start: None}
    visited = set()
    
    while frontier:
        frontier.sort(key=lambda x: x[1])
        current_node, _ = frontier.pop(0)
        
        if current_node in visited:
            continue

        visited.add(current_node)
        
        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = came_from[current_node]
            path.reverse()
            print(f"\nGoal found with A*. Path: {path}")
            return
        
        for neighbor, cost in graph[current_node].items():
            new_g_cost = g_costs[current_node] + cost
            f_cost = new_g_cost + heuristic[neighbor]
            
            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                came_from[neighbor] = current_node
                frontier.append((neighbor, f_cost))
        
        # Simulate real-time changes in edge costs dynamically at randomintervals
        if random.random() < 0.3: 
            update_edge_costs(graph)
            print("\nEdge costs updated dynamically!")
            print(graph)  
            time.sleep(1)

print("\nFollowing is the A* Search:")
a_star(graph, 'A', 'I')



Following is the A* Search:

Edge costs updated dynamically!
{'A': {'B': 1, 'C': 3}, 'B': {'D': 2, 'E': 7}, 'C': {'F': 6, 'G': 7}, 'D': {'H': 1}, 'E': {}, 'F': {'I': 1}, 'G': {}, 'H': {}, 'I': {}}

Edge costs updated dynamically!
{'A': {'B': 7, 'C': 8}, 'B': {'D': 4, 'E': 2}, 'C': {'F': 10, 'G': 8}, 'D': {'H': 9}, 'E': {}, 'F': {'I': 5}, 'G': {}, 'H': {}, 'I': {}}

Goal found with A*. Path: ['A', 'C', 'F', 'I']


In [3]:
import heapq

# Graph with edge costs (travel times)
graph = {
    'A': {'B': 2, 'C': 1},
    'B': {'D': 4, 'E': 3},
    'C': {'F': 1, 'G': 5},
    'D': {'H': 2},
    'E': {},
    'F': {'I': 6},
    'G': {},
    'H': {},
    'I': {}
}

heuristic = {
    'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 7,
    'F': 3, 'G': 6, 'H': 2, 'I': 0  
}

# Time windows for each node (earliest time, latest time)
time_windows = {
    'A': (0, 12), 
    'B': (1, 10),   
    'C': (1, 9),   
    'D': (4, 12),
    'E': (5, 14),  
    'F': (2, 10),   
    'G': (3, 13),  
    'H': (7, 16),
    'I': (9, 18)   
}

def greedy_bfs_with_time(graph, start, goal, heuristic, time_windows):
    pq = []  
    heapq.heappush(pq, (heuristic[start], start, 0))  # (heuristic, node, arrival_time)
    
    best_time = {node: float('inf') for node in graph}  
    best_time[start] = 0
    
    came_from = {start: None}

    while pq:
        _, current_node, current_time = heapq.heappop(pq)

        # If reached goal within its time window, reconstruct path
        if current_node == goal:
            if time_windows[goal][0] <= current_time <= time_windows[goal][1]:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                print(f"Goal reached. Optimal path: {path}")
                return
            else:
                continue  # Goal reached outside time window, ignore this path

        print(f"Visiting {current_node} at time {current_time}")

        for neighbor, travel_time in graph[current_node].items():
            arrival_time = current_time + travel_time

            # Adjust arrival time to fit within time window
            if arrival_time < time_windows[neighbor][0]:
                arrival_time = time_windows[neighbor][0]  # Wait until allowed

            # If arrival time is within time window and better than previous, update
            if time_windows[neighbor][0] <= arrival_time <= time_windows[neighbor][1] and arrival_time < best_time[neighbor]:
                best_time[neighbor] = arrival_time
                came_from[neighbor] = current_node
                heapq.heappush(pq, (heuristic[neighbor], neighbor, arrival_time))

    print("Goal not reachable within time constraints.")

print("\nOptimized Delivery Route with Time Constraints:")
greedy_bfs_with_time(graph, 'A', 'I', heuristic, time_windows)



Optimized Delivery Route with Time Constraints:
Visiting A at time 0
Visiting C at time 1
Visiting F at time 2
Goal reached. Optimal path: ['A', 'C', 'F', 'I']
